NOTES

1) Import web scraping - 17S? Vote counter? GM sheets? 

In [1]:
# imports

import pandas as pd


In [2]:
path = "DataTest.csv"

data = pd.read_csv(path)

In [3]:
data.head()

ID      Player Alignment Faction Outcome  First Hit  Last Hit  # of Hits  \
0   1        Meta       NaN             NaN        NaN       NaN        NaN   
1   2    Cracknut         G               L        1.0       1.0        1.0   
2   3  Edgedancer         G               L        2.0       2.0        1.0   
3   4   Mailliw73         G               L        4.0       4.0        1.0   
4   5      Aether         E               W        4.0       4.0        1.0   

  Death/\n Survival Inactive   GM  ... Broken     Role  \
0               NaN      NaN    Y  ...    NaN       GM   
1                 E      NaN  NaN  ...    NaN   Tineye   
2                 L      NaN  NaN  ...    NaN      Cop   
3                 L      NaN  NaN  ...    NaN  Regular   
4                 V      NaN  NaN  ...    NaN   Smoker   

                           Unnamed: 17  Unnamed: 18 Unnamed: 19 Unnamed: 20  \
0  Roles that are unique and don't fit          NaN  Unfinished   Alignment   
1                MR1/LG8/MR7 need work          NaN         QF7           0   
2                                  NaN          NaN         MR6           0   
3                                  NaN          NaN        LG12           0   
4                                  NaN          NaN         QF8           0   

  Unnamed: 21 Unnamed: 22 Unnamed: 23 Unnamed: 24  
0     Victory        Hits    Survival         NaN  
1           0          13           0         NaN  
2           0          25           0         NaN  
3           0           2           0         NaN  
4           0          22           0         NaN  

[5 rows x 25 columns]

CLEANING

In [4]:
data.columns

Index(['ID', 'Player', 'Alignment', 'Faction Outcome', 'First Hit', 'Last Hit',
       '# of Hits', 'Death/\n Survival', 'Inactive', 'GM', 'Mod', 'Spec',
       'Game Type', 'Game #', 'Unnamed: 14', 'Broken', 'Role', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [5]:
data1 = data[["Player", "Alignment", "Faction Outcome", "First Hit", "Last Hit", "# of Hits", 
             "Death/\n Survival", "Inactive", 'GM', 'Mod', 'Spec', 'Game Type', 'Game #', 
             'Unnamed: 14', 'Broken', 'Role']]
data1 = data1.rename(columns = {"Faction Outcome": "fOutcome", "First Hit": "Hit1", "Last Hit": "HitL",
                       "# of Hits": "HitNum", "Death/\n Survival": "Death", "Game Type": "format",
                       "Game #": "GameNum", "Unnamed: 14": "Game"})

data1.head()

Player Alignment fOutcome  Hit1  HitL  HitNum Death Inactive   GM  Mod  \
0        Meta       NaN      NaN   NaN   NaN     NaN   NaN      NaN    Y  NaN   
1    Cracknut         G        L   1.0   1.0     1.0     E      NaN  NaN  NaN   
2  Edgedancer         G        L   2.0   2.0     1.0     L      NaN  NaN  NaN   
3   Mailliw73         G        L   4.0   4.0     1.0     L      NaN  NaN  NaN   
4      Aether         E        W   4.0   4.0     1.0     V      NaN  NaN  NaN   

  Spec format  GameNum Game Broken     Role  
0  NaN     LG      1.0  LG1    NaN       GM  
1  NaN     LG      1.0  LG1    NaN   Tineye  
2  NaN     LG      1.0  LG1    NaN      Cop  
3  NaN     LG      1.0  LG1    NaN  Regular  
4  NaN     LG      1.0  LG1    NaN   Smoker

In [21]:
data2 = data1.fillna({"Alignment": 0, 
                       "fOutcome": 0, 
                      "Hit1": "-", 
                      "HitL": "-", 
                      "HitNum": "-", 
                      "Inactive":"N", 
                      "GM":"N", 
                      "Mod": "N", 
                      "Spec": "N", 
                      "Broken": "N", 
                      "Role": "-"})
print(len(data2))
data2.head()

5763


Player Alignment fOutcome Hit1 HitL HitNum Death Inactive GM Mod Spec  \
0        Meta         0        0                    NaN        N  Y   N    N   
1    Cracknut         G        L    1    1      1     E        N  N   N    N   
2  Edgedancer         G        L    2    2      1     L        N  N   N    N   
3   Mailliw73         G        L    4    4      1     L        N  N   N    N   
4      Aether         E        W    4    4      1     V        N  N   N    N   

  format  GameNum Game Broken     Role  
0     LG      1.0  LG1      N       GM  
1     LG      1.0  LG1      N   Tineye  
2     LG      1.0  LG1      N      Cop  
3     LG      1.0  LG1      N  Regular  
4     LG      1.0  LG1      N   Smoker

C:\Users\Public\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Empty DataFrame
Columns: [ID, Player, Alignment, fOutcome, Hit1, HitL, HitNum, Death, Inactive, GM, Mod, Spec, format, GameNum, Game, Broken, Role]
Index: []

In [24]:
# separate out mod/spec/GM into separate table, tbh
nonplayers_list = []
it = data2.iterrows()
i = 0
while i != 5672:
    curr = next(it)
    
    ifmod = curr[1]["Mod"]
    ifGM = curr[1]["GM"]
    ifspec = curr[1]["Spec"]
    
    if (ifmod == "Y") or (ifGM == "Y") or (ifspec == "Y"):
        nonplayers_list.append(curr[0])
    
    i += 1

print(len(nonplayers_list))


1445


In [25]:

data3 = data2.filter(["Player", "GM", "Mod", "Spec", "format", "GameNum", "Game", "Broken"], axis = 1)
nonplayers = data3.filter(nonplayers_list, axis = 0)

nonplayers.head()

# to clean further: combine GM/mod/spec into one column, don't need 'Game' probably

Player GM Mod Spec format  GameNum Game Broken
0     Meta  Y   N    N     LG      1.0  LG1      N
17   Windy  N   N    Y     LG      1.0  LG1      N
18   Weiry  N   N    Y     LG      1.0  LG1      N
19  Alvron  N   N    Y     LG      1.0  LG1      N
20    Meta  Y   N    N     LG      2.0  LG2      N

1445

In [42]:
# t2 = []
# t1 = nonplayers.iterrows()

# for k in range(1445):
#     curr = next(t1)

#     ifGM = curr[1]["GM"]
 
#     if (ifGM == "Y"):
#         t2.append(curr[0])
    
# gms = nonplayers.filter(t2, axis = 0)

# gms.tail()

Player GM Mod Spec format  GameNum  Game Broken
5058       Kynedath  Y   N    N     LG     70.0  LG70      N
5075     EXPERIENCE  Y   N    N     MR     45.0  MR45      N
5093          Straw  Y   N    N     LG     71.0  LG71      N
5094  Matrim's_Dice  Y   N    N     LG     71.0  LG71      N
5121        Elkanah  Y   N    N     QF     48.0  QF48      N

In [26]:

playerlist = [j for j in range(5763) if j not in nonplayers_list]
len(playerlist)

4318

In [45]:
players = data2.filter(playerlist, axis = 0)
players = players.drop(['GM', 'Mod', 'Spec'], axis = 1)

players.head()

Player Alignment fOutcome Hit1 HitL HitNum Death Inactive format  \
1    Cracknut         G        L    1    1      1     E        N     LG   
2  Edgedancer         G        L    2    2      1     L        N     LG   
3   Mailliw73         G        L    4    4      1     L        N     LG   
4      Aether         E        W    4    4      1     V        N     LG   
5     Claincy         E        W                0     S        N     LG   

   GameNum Game Broken         Role  
1      1.0  LG1      N       Tineye  
2      1.0  LG1      N          Cop  
3      1.0  LG1      N      Regular  
4      1.0  LG1      N       Smoker  
5      1.0  LG1      N  Vote Cancel

In [46]:
players.columns

Index(['Player', 'Alignment', 'fOutcome', 'Hit1', 'HitL', 'HitNum', 'Death',
       'Inactive', 'format', 'GameNum', 'Game', 'Broken', 'Role'],
      dtype='object')

In [50]:
players['Alignment'].value_counts()

G    2426
E     631
0     622
F     281
N     220
B      51
M      32
S      21
C      20
D      14
Name: Alignment, dtype: int64

In [47]:
g_players = players.groupby('Player')

In [48]:
g_players["format"].value_counts()

Player             format
18thShard          LG         2
A Budgie           LG         2
                   MR         2
                   QF         1
A Joe in the Bush  LG        30
                             ..
xxGaea             MR         6
                   LG         3
                   QF         3
yafeshan           LG         3
                   MR         1
Name: format, Length: 789, dtype: int64

In [49]:
players["Alignment"].value_counts()

G    2426
E     631
0     622
F     281
N     220
B      51
M      32
S      21
C      20
D      14
Name: Alignment, dtype: int64

In [ ]:
'Mailliw73', 'Alvron', 'A Joe in the Bush', 'Aonar',
'Kasimir', 'Haelbarde', 'Amanuensis', 'Orlok Tsubodai', 'Seonid',
'STINK', 'Straw', 'Young Bard', 'TheMightyLopen', 'Drake Marshall', 
'Arraenae', 'DeTess', 'BrightnessRadiant', 'Kidpen', 'StrikerEZ',
'Elkanah', 'Zillah', 'Kynedath', "Matrim's_Dice"